In [1]:
import datetime
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

list_titles,list_categories,list_stars,list_prices,is_in_stock = [],[],[],[],[]

def get_books(book_containers,html_soup):

    for j in range(0,len(book_containers)):
        list_prices.append(book_containers[j].find_all('div')[1].find_all('p')[0].text.split('£')[1])
                
        is_in_stock.append( 'Yes' if 'In stock' in book_containers[j].find_all('div')[1].find_all('p')[1].text else 'No')
                
        list_stars.append(book_containers[j].find_all('p')[0]['class'][1])
                
        list_titles.append(book_containers[j].find_all('div')[0].a.img['alt'])
                
        list_categories.append(html_soup.title.text.split('|')[0].strip())
      
    return

###-  Set the correct path where chrome webdriver is located
os.environ["WDPATH"] = "C:\\Users\pedro\Dropbox\pythonApplications\DataEngineering_project"
wdpath = os.environ["WDPATH"]
os.chdir("./subroutines")
from getCategories import get_categories
from checkWebdriver import check_webdriver
os.chdir('../')


__Checking if webdriver's path is OK__

In [2]:
if check_webdriver(wdpath):
    pass
else:
    exit()

Chrome Web Driver location is OK


__Getting all categories__

In [3]:
%%time
urls_to_scrape = []
if datetime.datetime.today().weekday()==0:
    try:
        urls_to_scrape = get_categories('http://books.toscrape.com/',wdpath)
    except RemoteDisconnected:
        print('\nNo internet connection!\n')
        exit()
    if urls_to_scrape != []:
        print('All categories escraped.')
        outFile = open('all_pages.txt','w')
        if outFile.mode == 'w':
            for link in urls_to_scrape:
                outFile.write(link+'\n')
            outFile.close()
        else:
            print('\nFile is not in writing mode\n')
            exit()
    else:
        print('Something went wrong when searching for categories.\n')
        exit()
else:
    print('\nPages not updated\n')
    inFile = open('all_pages.txt','r')
    if inFile.mode == 'r':
        urls_to_scrape = inFile.readlines()
    else:
        print('\nFile is not in reading mode\n')
        exit()



Pages not updated

Wall time: 0 ns


__Getting data from each page__

In [4]:
%%time
import grequests
from bs4 import BeautifulSoup

reqs = (grequests.get(link.strip()) for link in urls_to_scrape)
resp=grequests.imap(reqs, grequests.Pool(10))
for r in resp:
    html_soup = BeautifulSoup(r.text, 'html.parser')
    book_containers = html_soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3')
    get_books(book_containers,html_soup)

Wall time: 22.6 s


__Creating a DataFrame__

In [5]:
res = list_titles+list_categories+list_stars+list_prices+is_in_stock

if res ==[]:
    print('No data scraped.\n')
    exit()
else:
    df = pd.DataFrame({'Title':list_titles,'Category':list_categories,'Stars':list_stars,'Price':list_prices,'In Stock':is_in_stock})

    df['Stars'] = df['Stars'].map({'One':1,'Two':2,'Three':3,'Four':4,'Five':5})
    df['Title'] = df['Title'].apply(lambda x: x.replace('\x80\x99',"'") if '\x80\x99' in x else x)
    df['Title'] = df['Title'].apply(lambda x: x.replace('"',"'"))

    df.head()

 ### Checking if formatting is OK

In [6]:
## Checking if it's full of NaN
if df.isnull().sum().sum() == df.shape[0]*df.shape[1]:
    print('Only NaN values scraped. Something went wrong...\n')
    exit()

## Checking stars
nan_in_stars = df.Stars.isnull().sum()
if nan_in_stars == len(df):
    print('All stars scraped are NULL. Ending execution...\n')
    exit()
    
## Checking if there's something that differs from 'Yes', 'No' or 'Nan' in "In Stock" column
for count in df['In Stock'].dropna().value_counts().index:
    if df['In Stock'].isnull().sum() == len(df):
        print('All stars scraped are NULL. Ending execution...\n')
        exit()
    elif count !='Yes' and count !='No':
        print('Invalid value in "In Stock" column. Exiting...')
        exit()
        
## Checking if there's 'Nan' in title column
df['Title'].fillna('NULL', inplace=True)

### Creating a query to insert data into 'scraped' table

In [7]:
with open('./sql_files/insertInto_scrapedTable.sql','w', encoding="utf-8") as outFile:
    outFile.write('USE book_club_web_data;\n\n')
    outFile.write('INSERT INTO scraped(title,category,stars,price,is_in_stock) VALUES\n')
    for row in df.iloc:
        if row.name == df.iloc[-1].name:
            value = '("'+'","'.join(row.apply(lambda x: str(x)).values)+'");\n'
            value = value.replace('"NULL"','NULL')
            outFile.write(value)
        else:
            value = '("'+'","'.join(row.apply(lambda x: str(x)).values)+'"),\n'
            value = value.replace('"NULL"','NULL')
            outFile.write(value)
    outFile.close()